<a href="https://colab.research.google.com/github/Mongeese1/phyiscs-big-data/blob/master/gravwaves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math as math
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import scipy as sp
import scipy.special as special
from scipy.misc import derivative
import pandas as pd
import sympy as sym
from scipy.optimize import fmin
e=math.e
pi=math.pi

In [6]:
f = 1e-4
G = 6.67*10**(-11)
c = 3*10**8
neg=1j
# convert 10**30 kg to natural numbers
m_one = 10**30/(c**2/G)
# print(m_one)
m_two = 15**30/(c**2/G)
# convert 356 ly to natural numbers
# A = 356*24*60*60
# A = np.sqrt()
M = m_one+m_two
S_O = 6*10**(-49)
f_s = 20
f_0 = 70
l = f/f_0
t = 10
phic = 100
v=(pi*M*f)**(1/3)
n = m_one*m_two/(M**2) 
p = [phic, t, M, n]
low = 1
high = 10
# p = np.array(p)

a = [1, 0, (20/9*(743/336 + n*11/4)), -16*pi, 10*(3058673/1016064 + n*5429/1008 + (n**2) * 617/144)]
# a = np.array(a)
# zum = []
def Sum(z,a):
  z *=2
  j=0
  for i in range(len(a)):
    j += a[i]*(v**i)
  return j

def S(y,S_O):
  S = S_O*(y**(-4.14)) - 5*(y**2) + (111*(1-y**2 + ((y**4)/2))/(1+(y**2)/2))
  return S
# 
zum = Sum(1,a)

# psif = 2*pi * f * t - phic - pi/4 + (3/(128*n*(v**5)))*(zum)

# waveform = A*(f**(-7/6))*(e**(neg*psif))

def waveform1(f,phic,t,M,n,zum,v,A):
  # phic, t, M, n = p
  psif = 2*pi * f * t - phic - pi/4 + (3/(128*n*(v**5)))*(zum)
  waveform = A*(f**(-7/6))*(e**(neg*psif))
  return waveform
  

y = sym.symbols('y')
indefa = sym.integrate((f**(-7/6))/S(f,S_O),y)

defa = indefa.replace(y,high) - indefa.replace(y,low)

defa = float(defa)
A = np.sqrt(10 / 4 * defa)

print(A)
  
# print(fora)
# def deriv(x):
# def deriv(x):
#   phic, t, M , n = sym.symbols("phic t M n")
#   return sym.diff(waveform, phic)

# phic, t, M , n = sym.symbols("phic t M n", real=False)
# sym.diff(waveform, t)

def deriv(num,p,phic,t,M,n,zum,v):
  if num == 0:
    dphic = p[0]/1000
    pphic = p.copy()
    pphic[num] += dphic
    dy = waveform1(f,dphic,t,M,n,zum,v,A) - waveform1(f,phic,t,M,n,zum,v,A)
    return(dy/dphic)
  if num == 1:
    dt = p[1]/1000
    pt = p.copy()
    pt[num] += dt
    dy = waveform1(f,dt,t,M,n,zum,v,A) - waveform1(f,phic,t,M,n,zum,v,A)
    return(dy/dt)
  if num == 2:
    dM = p[2]/1000
    pM = p.copy()
    pM[num] += dM
    dy = waveform1(f,dM,t,M,n,zum,v,A) - waveform1(f,phic,t,M,n,zum,v,A)
    return(dy/dM)  
  if num == 3:
    dn = p[3]/1000
    pn = p.copy()
    pn[num] += dn
    dy = waveform1(f,dn,t,M,n,zum,v,A) - waveform1(f,phic,t,M,n,zum,v,A)
    return(dy/dn)
  else:
    print("Function accepts values of 0-3, where these values are which derivitave is being taken, 0=phic, 1=t, 2=M, 3=n")
    return
# deriv(0,p,phic,t,M,n,zum,v)



def create_fisher_matrix(increment,p,phic,t,M,n,zum,v,num,S_O):
    fisher_matrix = []
    for i in range(len(p)):
        fisher_matrix.append([])
        for j in range(len(p)):
            integral = 0
            for k in range(int(low*(1/increment)), int(high*(1/increment))):
                integral += np.real(deriv(i,p,phic,t,M,n,zum,v)*deriv(j,p,phic,t,M,n,zum,v)/S(k*increment,S_O))
            # print(integral)
            fisher_matrix[-1].append(4 * integral)
    return(fisher_matrix)

for i in range(4):
  matrix = create_fisher_matrix(0.01,p,phic,t,M,n,zum,v,i,S_O)
print(matrix)



inverse = np.linalg.inv(matrix)
print(inverse)

diag = np.sqrt(np.diag(inverse))
# print(np.sqrt(diag))

print(diag[3]/n)

96.99807296770113
[[-1237013886664047.5, -1.3685646148139928e+16, 16644112123.256577, -2.640063301544157e+22], [-1.3685646148139928e+16, -1.4373544888780525e+17, 140558222146.61057, -2.7602611970046446e+23], [16644112123.256577, 140558222146.61057, 23540.73163518978, 2.6404748189227562e+17], [-2.640063301544157e+22, -2.7602611970046446e+23, 2.6404748189227562e+17, -5.298594095017238e+29]]
[[-1.99975147e-01  6.03229318e-02  3.32769826e+03 -1.98025435e-08]
 [ 5.97636068e-02 -1.25293682e-02 -1.10100419e+03  3.00064210e-09]
 [ 3.33853232e+03 -1.11357979e+03 -5.34920680e+07  3.87108905e-04]
 [-1.95057685e-08  2.96650647e-09  3.81097561e-04 -3.83574923e-16]]
nan


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:125: RuntimeWarning: invalid value encountered in sqrt
